<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/PAPER_GDRNet_ELECTRICITY_Step4-NO-VAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import numpy as np
import scipy.stats as stats
from numpy import array
from numpy import hstack
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV, MultiTaskLassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import TimeSeriesSplit
from math import sqrt
from sklearn.linear_model import LinearRegression
from keras.models import Model
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import acf
from sklearn.ensemble import RandomForestRegressor
from numpy import arange
#-----------------------------------------------------GET DATA - X, Predicted Wndow from MLP, and Derived Labels, for the entire dataset that was gennerated through VAE-----------------------------------


#-----------for test dataset-----------------------------------------------------------------------
y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/ELEC_predicted_window.csv')
y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/ELEC_derived_window_label.csv')
x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/ELEC_test_data.csv')

n_features = 5
window_size = int(x_orig.shape[1]/n_features)
x = x_orig.reshape((x_orig.shape[0],window_size,n_features))

sample_size = 100
sample_size = x.shape[0]

#-----------------------------------------------CASES - pot window values and pick a few fixed window sizes -------------------------------------------



plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.title("Distribution before Transformation", fontsize=15)
sns.histplot(y_true, kde=True, color="red")
plt.subplot(1,2,2)

# Import libraries
import matplotlib.pyplot as plt
import numpy as np
fig = plt.figure(figsize =(10, 7))
# Creating plot
plt.boxplot(y_pred)
# show plot
plt.show()

fx_window1 = 5
fx_window2 = 10
fx_window3 = 15
fx_window4 = 20



################---------------------------METHOD 2- -GO BY RMSE----------------------------------------------------####################

def generate_small_seq(series, n_past, n_future):
  #
  # n_past ==> no of past observations -- OR -- sliding window
  #
  # n_future ==> no of future observations -- prediction variable y
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, 1] #We just take the active power as the prediction variable
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)



mse_list1 = list()
mse_list2 = list()
mse_list3 = list()
mse_list4 = list()
mse_listx = list()

min_window_list = list()
best_window_for_long_seq = list()
AIC = list()
n_fold = 6 # 5 fold plus 1 for test)
# evaluate a logistic regression model using k-fold cross-validation
n_future=1
predictions = list()


from numpy import mean
from numpy import std

best_window_for_long_seq.clear()

#window_list = [fx_window1, fx_window2, fx_window3, fx_window4]
########------------Window 1-------------------------


############## Window 1-----------------------------------
mse_list1.clear()
for i in range(sample_size) :
#iterate over the entire  long sequences of multivariate time series
  Total_small_seq_X, Total_small_seq_y = generate_small_seq(x[i,:,:], fx_window1 , n_future)
  Total_small_seq_X_reshaped = Total_small_seq_X.reshape(Total_small_seq_X.shape[0],Total_small_seq_X.shape[1]*Total_small_seq_X.shape[2])
  Total_small_seq_y_reshaped = Total_small_seq_y
  maxval2 = Total_small_seq_X_reshaped.shape[0]
  count_train2 = int(math.ceil(0.7*maxval2))
  Small_train_X = Total_small_seq_X_reshaped[0:count_train2]
  Small_train_y = Total_small_seq_y_reshaped[0:count_train2]
  Small_test_X =  Total_small_seq_X_reshaped[count_train2:]
  Small_test_y = Total_small_seq_y_reshaped[count_train2:]
  tscv = TimeSeriesSplit(n_splits=n_fold-1)
  #regressor = LassoCV(alphas=arange(0, 1, 0.01), cv=tscv, n_jobs=-1)
  regressor = LinearRegression()
  try:
    regressor.fit(Small_train_X, Small_train_y.ravel()) #get jth time series only
    if Small_test_X.shape[0] == 0:
      break
    y_predk = regressor.predict(Small_test_X)
    #print('alpha: %f' % regressor.alpha_)
    mse = mean_squared_error(Small_test_y, y_predk)
    #print(rmse)
    mse_list1.append(mse)
  except:
    mse_list1.append(9999)
    print('error')

mse_list1 = [i for i in mse_list1 if i != 9999]
mse1_avg = np.mean(mse_list1)
print(mse1_avg)


############## Window 2-----------------------------------
mse_list2.clear()
for i in range(sample_size) :
#iterate over the entire  long sequences of multivariate time series
  Total_small_seq_X, Total_small_seq_y = generate_small_seq(x[i,:,:], fx_window2 , n_future)
  Total_small_seq_X_reshaped = Total_small_seq_X.reshape(Total_small_seq_X.shape[0],Total_small_seq_X.shape[1]*Total_small_seq_X.shape[2])
  Total_small_seq_y_reshaped = Total_small_seq_y
  maxval2 = Total_small_seq_X_reshaped.shape[0]
  count_train2 = int(math.ceil(0.7*maxval2))
  Small_train_X = Total_small_seq_X_reshaped[0:count_train2]
  Small_train_y = Total_small_seq_y_reshaped[0:count_train2]
  Small_test_X =  Total_small_seq_X_reshaped[count_train2:]
  Small_test_y = Total_small_seq_y_reshaped[count_train2:]
  tscv = TimeSeriesSplit(n_splits=n_fold-1)
  #regressor = LassoCV(alphas=arange(0, 1, 0.01), cv=tscv, n_jobs=-1)
  regressor = LinearRegression()
  try:
    regressor.fit(Small_train_X, Small_train_y.ravel()) #get jth time series only
    if Small_test_X.shape[0] == 0:
      break
    y_predk = regressor.predict(Small_test_X)
    #print('alpha: %f' % regressor.alpha_)
    mse = mean_squared_error(Small_test_y, y_predk)
    #print(rmse)
    mse_list2.append(mse)
  except:
    mse_list2.append(9999)
    print('error')

mse_list2 = [i for i in mse_list2 if i != 9999]
mse2_avg = np.mean(mse_list2)
print(mse2_avg)

############## Window 3-----------------------------------
mse_list3.clear()
for i in range(sample_size) :
#iterate over the entire  long sequences of multivariate time series
  Total_small_seq_X, Total_small_seq_y = generate_small_seq(x[i,:,:], fx_window3 , n_future)
  Total_small_seq_X_reshaped = Total_small_seq_X.reshape(Total_small_seq_X.shape[0],Total_small_seq_X.shape[1]*Total_small_seq_X.shape[2])
  Total_small_seq_y_reshaped = Total_small_seq_y
  maxval2 = Total_small_seq_X_reshaped.shape[0]
  count_train2 = int(math.ceil(0.7*maxval2))
  Small_train_X = Total_small_seq_X_reshaped[0:count_train2]
  Small_train_y = Total_small_seq_y_reshaped[0:count_train2]
  Small_test_X =  Total_small_seq_X_reshaped[count_train2:]
  Small_test_y = Total_small_seq_y_reshaped[count_train2:]
  tscv = TimeSeriesSplit(n_splits=n_fold-1)
  #regressor = LassoCV(alphas=arange(0, 1, 0.01), cv=tscv, n_jobs=-1)
  regressor = LinearRegression()
  try:
    regressor.fit(Small_train_X, Small_train_y.ravel()) #get jth time series only
    if Small_test_X.shape[0] == 0:
      break
    y_predk = regressor.predict(Small_test_X)
    #print('alpha: %f' % regressor.alpha_)
    mse = mean_squared_error(Small_test_y, y_predk)
    #print(rmse)
    mse_list3.append(mse)
  except:
    mse_list3.append(9999)
    print('error')

mse_list3 = [i for i in mse_list3 if i != 9999]
mse3_avg = np.mean(mse_list3)
print(mse3_avg)

############## Window 4-----------------------------------
mse_list4.clear()
for i in range(sample_size) :
#iterate over the entire  long sequences of multivariate time series
  Total_small_seq_X, Total_small_seq_y = generate_small_seq(x[i,:,:], fx_window4 , n_future)
  Total_small_seq_X_reshaped = Total_small_seq_X.reshape(Total_small_seq_X.shape[0],Total_small_seq_X.shape[1]*Total_small_seq_X.shape[2])
  Total_small_seq_y_reshaped = Total_small_seq_y
  maxval2 = Total_small_seq_X_reshaped.shape[0]
  count_train2 = int(math.ceil(0.7*maxval2))
  Small_train_X = Total_small_seq_X_reshaped[0:count_train2]
  Small_train_y = Total_small_seq_y_reshaped[0:count_train2]
  Small_test_X =  Total_small_seq_X_reshaped[count_train2:]
  Small_test_y = Total_small_seq_y_reshaped[count_train2:]
  tscv = TimeSeriesSplit(n_splits=n_fold-1)
  #regressor = LassoCV(alphas=arange(0, 1, 0.01), cv=tscv, n_jobs=-1)
  regressor = LinearRegression()
  try:
    regressor.fit(Small_train_X, Small_train_y.ravel()) #get jth time series only
    if Small_test_X.shape[0] == 0:
      break
    y_predk = regressor.predict(Small_test_X)
    #print('alpha: %f' % regressor.alpha_)
    mse = mean_squared_error(Small_test_y, y_predk)
    #print(rmse)
    mse_list4.append(mse)
  except:
    mse_list4.append(9999)
    print('error')

mse_list4 = [i for i in mse_list4 if i != 9999]
mse4_avg = np.mean(mse_list4)
print(mse4_avg)

############## NOW FOR PREDICTED WINDOW -----------------------------------
mse_listx.clear()
for i in range(sample_size) :
#iterate over the entire  long sequences of multivariate time series
  Total_small_seq_X, Total_small_seq_y = generate_small_seq(x[i,:,:], int(y_pred[i]) , n_future)
  Total_small_seq_X_reshaped = Total_small_seq_X.reshape(Total_small_seq_X.shape[0],Total_small_seq_X.shape[1]*Total_small_seq_X.shape[2])
  Total_small_seq_y_reshaped = Total_small_seq_y
  maxval2 = Total_small_seq_X_reshaped.shape[0]
  count_train2 = int(math.ceil(0.7*maxval2))
  Small_train_X = Total_small_seq_X_reshaped[0:count_train2]
  Small_train_y = Total_small_seq_y_reshaped[0:count_train2]
  Small_test_X =  Total_small_seq_X_reshaped[count_train2:]
  Small_test_y = Total_small_seq_y_reshaped[count_train2:]
  tscv = TimeSeriesSplit(n_splits=n_fold-1)
  #regressor = LassoCV(alphas=arange(0, 1, 0.01), cv=tscv, n_jobs=-1)
  regressor = LinearRegression()
  try:
    regressor.fit(Small_train_X, Small_train_y.ravel()) #get jth time series only
    if Small_test_X.shape[0] == 0:
      break
    y_predk = regressor.predict(Small_test_X)
    #print('alpha: %f' % regressor.alpha_)
    mse = mean_squared_error(Small_test_y, y_predk)
    #print(rmse)
    mse_listx.append(mse)
  except:
    mse_listx.append(9999)
    print('error')

mse_listx = [i for i in mse_listx if i != 9999]
msex_avg = np.mean(mse_listx)
print(msex_avg)


############## Overall-----------------------------------
mse_max = np.array([np.max(mse_list1),np.max(mse_list2),np.max(mse_list3),np.max(mse_list4)]).max()

norm_rmse_list1 = sqrt(mse1_avg/mse_max)
norm_rmse_list2 = sqrt(mse2_avg/mse_max)
norm_rmse_list3 = sqrt(mse3_avg/mse_max)
norm_rmse_list4 = sqrt(mse4_avg/mse_max)
norm_rmse_listx = sqrt(msex_avg/mse_max)
print(norm_rmse_list1)
print(norm_rmse_list2)
print(norm_rmse_list3)
print(norm_rmse_list4)
print(norm_rmse_listx)



df.to_csv('/content/drive/MyDrive/PHD/2024/PREDICTION_COMPARISON/Electricity_mse_comparison.csv', index = False)


0.18805389126996985
0.1282281253287555
0.11920464660821116
0.1347701906171697
0.1064841542470815


In [ ]:
from google.colab import drive
drive.mount('/content/drive')